### 5. TFIDF deep learning classifier

In [9]:
import pandas as pd

lyrics = pd.read_csv('../dataset/Lyrics_en_clean.csv', index_col=['Band','Song'])
genres = pd.read_csv('../dataset/artists_final_ohe.csv', index_col=['Band'])
df = pd.merge(lyrics, genres, on=['Band'])
del lyrics
del genres

y_train = df.drop(['Lyrics'], axis=1)

In [10]:
import gensim.corpora as corpora
from gensim.models import TfidfModel
import pickle

#LOAD LYRICS from disk
lyrics = list()
with open ('../dataset/lemma_lyrics', 'rb') as fp:
    lyrics = pickle.load(fp)

id2word = corpora.Dictionary.load("../dataset/lemma_lyrics_dict")
bow_corpus = list()

for lyric in lyrics:
    bow_corpus.append(id2word.doc2bow(lyric))

tfidf = TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

In [11]:
from gensim.matutils import corpus2csc
tfidf_corpus = tfidf[bow_corpus]
tfidf_scipy = corpus2csc(tfidf_corpus)
del tfidf_corpus
del tfidf
del bow_corpus
del lyrics

In [13]:
from sklearn.model_selection import train_test_split

X_train = tfidf_scipy.transpose()
X_train,X_valid,y_train,y_valid = train_test_split(X_train, y_train, test_size=0.4)

print('Train shape: ',X_train.shape,',',y_train.shape)
print('Valid shape: ',X_valid.shape,',',y_valid.shape)

del df

Train shape:  (200199, 181260) , (200199, 646)
Valid shape:  (133467, 181260) , (133467, 646)


In [12]:
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from keras.utils import Sequence

class My_Generator(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(self.x.shape[0] / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_x.toarray(), np.array(batch_y)

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

#from sklearn.manifold import TSNE
train_size = X_train.shape
batch_size = 16
my_training_batch_generator = My_Generator(X_train, y_train, batch_size)
num_training_samples = X_train.shape[0]
num_epochs = 3
my_validation_batch_generator = My_Generator(X_valid, y_valid, batch_size)
num_validation_samples = X_valid.shape[0]

train_size = y_train.shape[0] 
model_conv = Sequential()
model_conv.add(Embedding(input_dim=train_size, output_dim=100))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
model_conv.add(Dense(y_train.shape[1], activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
model_conv.summary()

#model_conv.fit(X_train_tfidf, y_train, validation_split=0.2, epochs = 3, batch_size=16)
model_conv.fit_generator(generator=my_training_batch_generator,
                      steps_per_epoch=(num_training_samples // batch_size),
                      epochs=num_epochs,
                      verbose=1,
                      validation_data=my_validation_batch_generator,
                      validation_steps=(num_validation_samples // batch_size),
                      #use_multiprocessing=True,
                      workers=8,
                      max_queue_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         20019900  
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 646)               65246     
Total params: 20,183,210
Trainable params: 20,183,210
Non-trainable params: 0
________________________________________________________________

KeyboardInterrupt: 